In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix


In [2]:
dataframe = pd.read_csv('forestfires.csv')
dataframe.head()

,X,Y,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area
0,7,5,mar,fri,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,0.0
1,7,4,oct,tue,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,0.0
2,7,4,oct,sat,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,0.0
3,8,6,mar,fri,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,0.0
4,8,6,mar,sun,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,0.0


In [3]:
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 517 entries, 0 to 516
Data columns (total 13 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   X       517 non-null    int64  
 1   Y       517 non-null    int64  
 2   month   517 non-null    object 
 3   day     517 non-null    object 
 4   FFMC    517 non-null    float64
 5   DMC     517 non-null    float64
 6   DC      517 non-null    float64
 7   ISI     517 non-null    float64
 8   temp    517 non-null    float64
 9   RH      517 non-null    int64  
 10  wind    517 non-null    float64
 11  rain    517 non-null    float64
 12  area    517 non-null    float64
dtypes: float64(8), int64(3), object(2)
memory usage: 52.6+ KB


In [4]:
dataframe.describe()

,X,Y,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area
count,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000
mean,4.669246,4.299807,90.644681,110.872340,547.940039,9.021663,18.889168,44.288201,4.017602,0.021663,12.847292
std,2.313778,1.229900,5.520111,64.046482,248.066192,4.559477,5.806625,16.317469,1.791653,0.295959,63.655818
min,1.000000,2.000000,18.700000,1.100000,7.900000,0.000000,2.200000,15.000000,0.400000,0.000000,0.000000
25%,3.000000,4.000000,90.200000,68.600000,437.700000,6.500000,15.500000,33.000000,2.700000,0.000000,0.000000
50%,4.000000,4.000000,91.600000,108.300000,664.200000,8.400000,19.300000,42.000000,4.000000,0.000000,0.520000
75%,7.000000,5.000000,92.900000,142.400000,713.900000,10.800000,22.800000,53.000000,4.900000,0.000000,6.570000
max,9.000000,9.000000,96.200000,291.300000,860.600000,56.100000,33.300000,100.000000,9.400000,6.400000,1090.840000


In [5]:
dataframe['month'].unique()

array(['mar', 'oct', 'aug', 'sep', 'apr', 'jun', 'jul', 'feb', 'jan',
       'dec', 'may', 'nov'], dtype=object)

In [6]:
dataframe['day'].unique()

array(['fri', 'tue', 'sat', 'sun', 'mon', 'wed', 'thu'], dtype=object)

In [7]:
dataframe['day'] =dataframe['day'].apply(func = lambda x: ['sun', 'mon','tue', 'wed', 'thu','fri',  'sat'].index(x)+1)

In [8]:
dataframe['month'] =dataframe['month'].apply(func = lambda x: [ 'jan', 'feb','mar','apr','may', 'jun', 'jul', 'aug', 'sep', 'oct',  'nov', 'dec' ].index(x)+1)

In [9]:
dataframe.corr()

,X,Y,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area
X,1.000000,0.539548,-0.065003,0.013037,-0.021039,-0.048384,-0.085916,0.006210,-0.051258,0.085223,0.018798,0.065387,0.063385
Y,0.539548,1.000000,-0.066292,-0.013085,-0.046308,0.007782,-0.101178,-0.024488,-0.024103,0.062221,-0.020341,0.033234,0.044873
month,-0.065003,-0.066292,1.000000,-0.037469,0.291477,0.466645,0.868698,0.186597,0.368842,-0.095280,-0.086368,0.013438,0.056496
day,0.013037,-0.013085,-0.037469,1.000000,0.073597,0.028697,0.001913,0.035926,0.032233,-0.083318,-0.004013,-0.024119,0.048359
FFMC,-0.021039,-0.046308,0.291477,0.073597,1.000000,0.382619,0.330512,0.531805,0.431532,-0.300995,-0.028485,0.056702,0.040122
DMC,-0.048384,0.007782,0.466645,0.028697,0.382619,1.000000,0.682192,0.305128,0.469594,0.073795,-0.105342,0.074790,0.072994
DC,-0.085916,-0.101178,0.868698,0.001913,0.330512,0.682192,1.000000,0.229154,0.496208,-0.039192,-0.203466,0.035861,0.049383
ISI,0.006210,-0.024488,0.186597,0.035926,0.531805,0.305128,0.229154,1.000000,0.394287,-0.132517,0.106826,0.067668,0.008258
temp,-0.051258,-0.024103,0.368842,0.032233,0.431532,0.469594,0.496208,0.394287,1.000000,-0.527390,-0.227116,0.069491,0.097844
RH,0.085223,0.062221,-0.095280,-0.083318,-0.300995,0.073795,-0.039192,-0.132517,-0.527390,1.000000,0.069410,0.099751,-0.075519


In [10]:
X_data, Y_data = dataframe.iloc[:,0:len(dataframe.columns)-1], dataframe.iloc[:,len(dataframe.columns)-1]

In [11]:
ss =StandardScaler()
X_data = ss.fit_transform(X_data)

In [12]:
cross_val_score(estimator=LinearRegression(),X = X_data,y = Y_data)

array([ 0.        , -9.44980553, -0.04786842, -0.48791711,  0.01095213])

In [13]:
cross_val_score(estimator=DecisionTreeRegressor(),X = X_data,y = Y_data)

array([  0.        , -61.53716856,  -0.08292495,  -1.27357652,
        -0.4087252 ])

In [14]:
cross_val_score(estimator=RandomForestRegressor(),X = X_data,y = Y_data)

array([  0.        , -35.50946514,  -0.07588941,  -1.37167072,
        -0.06687869])

In [15]:
 X_train, X_test, y_train, y_test = train_test_split( X_data, Y_data, test_size=0.25, random_state=42)

In [16]:
lr = LinearRegression()
lr.fit(X_train,y_train)
lr.score(X_test,y_test)

0.007687478249823965

In [17]:
dt = DecisionTreeRegressor()
dt.fit(X_train,y_train)
dt.score(X_test,y_test)

-0.1840081052624496

In [18]:
rfc = RandomForestRegressor()
rfc.fit(X_train,y_train)
rfc.score(X_test,y_test)

-0.00125618915858694

# regression models prediction are poor because of biased data and finding whether area is burnt or not using classification

In [19]:
dataframe['area'] = dataframe['area'].apply(lambda x: 1 if x>0 else 0)

In [20]:
dataframe['area'].unique()

array([0, 1], dtype=int64)

In [21]:
X_data, Y_data = dataframe.iloc[:,0:len(dataframe.columns)-1], dataframe.iloc[:,len(dataframe.columns)-1]
X_train, X_test, y_train, y_test = train_test_split( X_data, Y_data, test_size=0.25, random_state=42)

In [22]:
dt = DecisionTreeClassifier(random_state=12)
dt.fit(X_train,y_train)
dt.score(X_train,y_train)

0.9922480620155039

In [23]:
dt.score(X_test,y_test)

0.5153846153846153

In [24]:
rf = RandomForestClassifier(random_state=12)
rf.fit(X_train,y_train)
rf.score(X_train,y_train)

0.9922480620155039

In [25]:
rf.score(X_test,y_test)


0.6076923076923076

In [26]:
confusion_matrix(y_test,rf.predict(X_test))

array([[33, 28],
       [23, 46]], dtype=int64)